In [1]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 16 14:09:38 2018

@author: Aradhya
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Feb 26 00:25:32 2018

@author: Admin
"""

#using HOG , division of features and then putting SVM
#from train.py, windows.py, features.py
import numpy as np
import cv2
from skimage.feature import hog
import pickle
import os
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from settings import *
from sklearn.neighbors import KNeighborsClassifier
from skimage import exposure
from skimage import feature
from imutils import paths
import argparse
import imutils
from sklearn.preprocessing import StandardScaler
#from train1 import *
# import support vector classifier
from sklearn.svm import LinearSVC# "Support Vector Classifier"
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import classification
#from classification import *
#from plot_confusion_matrix1 import *
#from confusion_matrix1 import *
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
#from pyimagesearch.helpers import pyramid
#from pyimagesearch.helpers import sliding_window

#import PIL
#from PIL import Image

In [3]:
#the following code is for finding HOG using code #1  : features.py
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False,transform_sqrt=True, feature_vec=True):
	"""
	Return HOG features and visualization (optionally)
	"""
	if vis == True:
		features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
			cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
			visualize=True, feature_vector=False)
		return features, hog_image
	else:
		features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
			cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
			visualize=False, feature_vector=feature_vec)
		return features

In [4]:
# Define a function that takes an image,
# start and stop positions in both x and y,
# window size (x and y dimensions),
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None],
					xy_window=(100,84), xy_overlap=(0.5, 0.5)):
	# If x and/or y start/stop positions not defined, set to image size
	if x_start_stop[0] == None:
		x_start_stop[0] = 0
	if x_start_stop[1] == None:
		x_start_stop[1] = img.shape[1]
	if y_start_stop[0] == None:
		y_start_stop[0] = 0
	if y_start_stop[1] == None:
		y_start_stop[1] = img.shape[0]
	# Compute the span of the region to be searched
	xspan = x_start_stop[1] - x_start_stop[0]
	yspan = y_start_stop[1] - y_start_stop[0]
	# Compute the number of pixels per step in x/y
	nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
	ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
	# Compute the number of windows in x/y
	nx_windows = np.int(xspan/nx_pix_per_step) - 1
	ny_windows = np.int(yspan/ny_pix_per_step) - 1
	# Initialize a list to append window positions to
	window_list = []
	# Loop through finding x and y window positions
	# Note: you could vectorize this step, but in practice
	# you'll be considering windows one by one with your
	# classifier, so looping makes sense
	for ys in range(ny_windows):
		for xs in range(nx_windows):
			# Calculate window position
			startx = xs*nx_pix_per_step + x_start_stop[0]
			endx = startx + xy_window[0]
			starty = ys*ny_pix_per_step + y_start_stop[0]
			endy = starty + xy_window[1]

			# Append window position to list
			window_list.append(((startx, starty), (endx, endy)))
	# Return the list of windows
	return window_list

In [5]:
color_space = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 16  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [300, 500] # Min and max in y to search in slide_window()

In [6]:
# Define a function you will pass an image
# and the list of windows to be searched (output of slide_windows())
#def search_windows(img, windows, clf, scaler, color_space='RGB',orient=9,pix_per_cell=16, cell_per_block=2):
def search_windowfeature(image2,color_space='RGB',orient=9,pix_per_cell=16, cell_per_block=2):       
    
#    for (x, y, window) in sliding_window(image2, stepSize=32, windowSize=(winW, winH)):            #    on_windows = []  	#1) Create an empty list to receive positive detection windows
#    for window in windows:  #2) Iterate over all windows in the list
#        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (100,84))   #3) Extract the test window from original image
    # loop over the sliding window for each layer of the pyramid
#        if window.shape[0] != winH or window.shape[1] != winW: # if the window does not meet our desired window size, ignore it
#            continue
        test_img = cv2.cvtColor(window,cv2.COLOR_RGB2GRAY)    
        hogfeaturestest, hog_imagetest = get_hog_features(test_img, orient,pix_per_cell, cell_per_block,vis=True,transform_sqrt=True,feature_vec=True,)
        featuresizetest=hogfeaturestest.size
        hogfeaturestest=np.reshape(hogfeaturestest,(featuresizetest,),order='C')
        hogfeaturestest= hogfeaturestest.reshape(1,-1)	# THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
        return hogfeaturestest    

#        testfeaturesh = X_scaler.transform(hogfeaturestest) 
#        prediction =svm_model_linear.predict(testfeaturesh) 
#        if prediction==1:
#            on_windows.append(window)
#        clone = image2.copy()
#        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#        cv2.imshow("Window", clone)
#		cv2.waitKey(1)
#		time.sleep(0.025)
		#5) Scale extracted features to be fed to classifier
#		test_features = scaler.transform(np.array(features).reshape(1, -1))
       
		#6) Predict using your classifier
#		prediction = clf.predict(test_features)
        
#        
#		#7) If positive (prediction == 1) then save the window
#        if prediction == 1:
#            on_windows.append(window)
#	#8) Return windows for positive detections
#    return on_windows

In [7]:
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=2):
	# Make a copy of the image
	imcopy = np.copy(img)
	# Iterate through the bounding boxes
	for bbox in bboxes:
		# Draw a rectangle given bbox coordinates
		cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
	# Return the image copy with boxes drawn
	return imcopy

In [8]:
def sliding_window(image, stepSize, windowSize):
	# slide a window across the image
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# yield the current window
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [9]:
if __name__ == '__main__':
#        positive='E:/Ankush Sir Work/PH.D_2017/Code/training/sample_2.JPG'
    posfeatures=[]
    h1=[]
    positive= 'C:/Users/hp/Desktop/Mod1-IITR/fishplate/pos/'
    for image_file in os.listdir(positive):
        image = mpimg.imread(os.path.join(positive,image_file))
#        image = cv2.imread(os.path.join(positive,image_file))
#        image = cv2.imread(positive)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        gray=np.resize(gray,(100,84))
#        gray=gray.reshape((50,50))
        
#    one method for finding hog............. !!
        hogfeatures, hog_image = get_hog_features(gray, orient=9,pix_per_cell=16, cell_per_block=2,vis=True,transform_sqrt=True,feature_vec=True,)
        featuresize=hogfeatures.size
        hogfeatures=np.reshape(hogfeatures,(featuresize,),order='C')
        hogfeatures= hogfeatures.reshape(1,-1)
#        fig = plt.figure()
#        plt.subplot(121)
#        plt.imshow(image)
#        plt.title('Example positive Image')
#        plt.subplot(122)
#        plt.imshow(hog_image, cmap='gray')
#        plt.title('HOG Visualization')
#        plt.show()
        posfeatures.append(hogfeatures)
#        tpos=get_hogfeaturesmain(positive)
        tpos=np.concatenate(posfeatures)  
        h1.append(np.concatenate(posfeatures))

#for negative examples
    negfeatures=[]
    h2=[]
    negative= 'C:/Users/hp/Desktop/Mod1-IITR/fishplate/neg/'
    for image_file1 in os.listdir(negative):
        image1 = mpimg.imread(os.path.join(negative,image_file1))
#        image1 = cv2.imread(os.path.join(negative,image_file1))
#        image = cv2.imread(positive)
        gray1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
        gray1=np.resize(gray1,(100,84))
#        gray=gray.reshape((50,50))
        
#    one method for finding hog............. !!
        hogfeatures1, hog_image1 = get_hog_features(gray1, orient=9,pix_per_cell=16, cell_per_block=2,vis=True,transform_sqrt=True,feature_vec=True,)
        featuresize1=hogfeatures1.size
        hogfeatures1=np.reshape(hogfeatures1,(featuresize1,),order='C')
        hogfeatures1= hogfeatures1.reshape(1,-1)
#        fig = plt.figure()
#        plt.subplot(121)
#        plt.imshow(image)
#        plt.title('Example positive Image')
#        plt.subplot(122)
#        plt.imshow(hog_image, cmap='gray')
#        plt.title('HOG Visualization')
#        plt.show()
        negfeatures.append(hogfeatures1)
#        tpos=get_hogfeaturesmain(positive)
        t=np.concatenate(negfeatures)  
        h2.append(np.concatenate(negfeatures))
    
#        t1=np.append(negfeatures)               TypeError: append() takes at least 2 arguments (1 given)
#        t3=negfeatures.extend(negfeatures)
#    h2.append(np.concatenate(negfeatures))    
#    
#SVM classification 
svc = LinearSVC()
X_scaler = StandardScaler() 
#train1(tpos,t,svc,X_scaler)
#"""
X = np.vstack((tpos,t))
	# Fit a per-column scaler
X_scaler.fit(X)
	# Apply the scaler to X
scaled_X = X_scaler.transform(X)

	# Define the labels vector
y = np.hstack((np.ones(len(tpos)), np.zeros(len(t))))
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
		scaled_X, y, test_size=0.30, random_state=rand_state)
    # Check the training time for the SVC
time1=time.time()
svm_model_linear=svc.fit(X_train, y_train)
#svc.fit(X_train, y_train)
t2 = time.time()

print(round(t2-time1, 15), 'Seconds to train SVC...')
svm_predictions = svm_model_linear.predict(X_test)
#svm_predictions = svc.predict(X_test)
print('Test Accuracy of SVC = ',svm_model_linear.score(X_test, y_test))  
#print('Test Accuracy of SVC = ',svc.score(X_test, y_test))  
cm = confusion_matrix(y_test, svm_predictions)
print('confusion matrix is,',cm)
lentest=len(y_test)
tp=float(cm[0,0])
fp=float(cm[0,1])
fn=float(cm[1,0])
tn=float(cm[1,1])
tpr=(tp/(tp+fn))
   
fpr=(fp/(fp+tn))
acc=((tp+tn)/lentest)
print('TRP=',tpr)
print('FPR=',fpr)
print('ACC=',acc)

#cv2.namedWindow("detected", cv2.WINDOW_NORMAL)    #for fitting the image normally into the window. from TemplateMatching.py
# Display predictions on all test_images
imdir = 'C:/Users/hp/Desktop/Mod1-IITR/test/'
xcord=[]
ycord1=[]
(winW, winH) = (100,84)
for image_file in os.listdir(imdir):
    image2 = mpimg.imread(os.path.join(imdir, image_file))
    #    image2 = cv2.imread(os.path.join(imdir, image_file))
#    image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)    
#    draw_image1 = np.copy(image2)
#    for resized in pyramid(image2, scale=1.5):    # loop over the image pyramid
    on_windows=[]
    pred=[]
    clone = image2.copy()
#    hot_windows =search_windowfeature(image2,color_space=color_space,orient=orient, pix_per_cell=pix_per_cell,cell_per_block=cell_per_block) 
    for (x, y, window) in sliding_window(image2, stepSize=32, windowSize=(winW, winH)):  # loop over the sliding window for each layer of the pyramid
        if window.shape[0] != winH or window.shape[1] != winW: # if the window does not meet our desired window size, ignore it
            continue
        windowstestfeat = search_windowfeature(window,color_space=color_space,orient=orient, pix_per_cell=pix_per_cell,cell_per_block=cell_per_block)  	# THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
        testfeaturesh = X_scaler.transform(windowstestfeat) 
        prediction =svm_model_linear.predict(testfeaturesh) 
        pred.append(prediction)
        if prediction==1:
            on_windows.append(window)
#            cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#            cv2.imshow("Window",clone)
#            xcord.append(x)
            xcord.append(x)
            ycord1.append(y)
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
        cv2.imshow("Window",clone)
        k=cv2.waitKey(1)
#        k = cv2.waitKey(0) & 0xFF
#        http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html
        
        time.sleep(0.025)
#            x.append(x)
#            x.concatenate(x)
    
#            ty=np.concatenate(y)
            
#    window_img = draw_boxes(draw_image1,on_windows, color=(0, 0, 255), thick=6)
#    plt.imshow(window_img)
#    plt.show()    
#        cv2.rectangle(draw_image1, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#        cv2.imshow("Detected",draw_image1)
#window_img = draw_boxes(image2,hot_windows, color=(0, 0, 255), thick=6)
#    plt.imshow(window_img)
#    plt.show()
#    
#    cv2.waitKey(1)
#    time.sleep(0.025)
           
#        windowstestfeat = search_windowfeature(window,color_space=color_space,orient=orient, pix_per_cell=pix_per_cell,cell_per_block=cell_per_block)  	# THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
#		# MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
#		# WINDOW
#        testfeaturesh = X_scaler.transform(windowstestfeat) 
#        prediction =svm_model_linear.predict(testfeaturesh) 
#        if prediction==1:
#            cv2.rectangle(draw_image1, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
##   return draw_image1 

#plt.imshow(draw_image1)
#plt.show()
#		# since we do not have a classifier, we'll just draw the window
#		clone = resized.copy()
#		cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#		cv2.imshow("Window", clone)
#		cv2.waitKey(1)
#		time.sleep(0.025)
    


## https://github.com/georgesung/vehicle_detection_hog_svm
##https://www.pyimagesearch.com/2014/11/10/histogram-oriented-gradients-object-detection/

0.002120733261108 Seconds to train SVC...
Test Accuracy of SVC =  0.8333333333333334
confusion matrix is, [[12  0]
 [ 4  8]]
TRP= 0.75
FPR= 0.0
ACC= 0.8333333333333334


ValueError: X has 1280 features, but this StandardScaler is expecting 720 features as input.